In [1]:
server = "opendata.dwd.de"
user   = "anonymous"
passwd = ""

In [2]:
# The topic of interest.
topic_dir = "/hourly/precipitation/historical/"
#topic_dir = "/annual/kl/historical/"

# This is the search pattern common to ALL station description file names 
station_desc_pattern = "_Beschreibung_Stationen.txt"

# Below this directory tree node all climate data are stored.
ftp_climate_data_dir = "/climate_environment/CDC/observations_germany/climate/"
ftp_dir =  ftp_climate_data_dir + topic_dir

In [3]:
local_ftp_dir         = "../data/original/DWD/"      # Local directory to store local ftp data copies, the local data source or input data. 
local_ftp_station_dir = local_ftp_dir + topic_dir # Local directory where local station info is located
local_ftp_ts_dir      = local_ftp_dir + topic_dir # Local directory where time series downloaded from ftp are located

local_generated_dir   = "../data/generated/DWD/" # The generated of derived data in contrast to local_ftp_dir
local_station_dir     = local_generated_dir + topic_dir # Derived station data, i.e. the CSV file
local_ts_merged_dir   = local_generated_dir + topic_dir # Parallelly merged time series, wide data frame with one TS per column
local_ts_appended_dir = local_generated_dir + topic_dir # Serially appended time series, long data frame for QGIS TimeManager Plugin


In [4]:
print(local_ftp_dir)
print(local_ftp_station_dir)
print(local_ftp_ts_dir)
print()
print(local_generated_dir)
print(local_station_dir)
print(local_ts_merged_dir)
print(local_ts_appended_dir)

../data/original/DWD/
../data/original/DWD//hourly/precipitation/historical/
../data/original/DWD//hourly/precipitation/historical/

../data/generated/DWD/
../data/generated/DWD//hourly/precipitation/historical/
../data/generated/DWD//hourly/precipitation/historical/
../data/generated/DWD//hourly/precipitation/historical/


In [5]:
import os
os.makedirs(local_ftp_dir,exist_ok = True) # it does not complain if the dir already exists.
os.makedirs(local_ftp_station_dir,exist_ok = True)
os.makedirs(local_ftp_ts_dir,exist_ok = True)

os.makedirs(local_generated_dir,exist_ok = True)
os.makedirs(local_station_dir,exist_ok = True)
os.makedirs(local_ts_merged_dir,exist_ok = True)
os.makedirs(local_ts_appended_dir,exist_ok = True)

In [6]:
import ftplib
ftp = ftplib.FTP(server)
res = ftp.login(user=user, passwd = passwd)
print(res)

230 Login successful.


In [7]:
#Importing the required packages
import os
import ftplib
import pandas as pd
import codecs
import seaborn as sns
from datetime import datetime
from zipfile import ZipFile
from matplotlib import *
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
#from adjustText import adjust_text
from sklearn.linear_model import LinearRegression
import matplotlib.colors as mcolors

In [8]:
!pip install seaborn

In [9]:
#generate data frame from ftp directories/files
def gen_df_from_ftp_dir_listing(ftp, ftpdir):
    lines = []
    flist = []
    try:
        # issue the command LIST in the FTP connection 
        res = ftp.retrlines("LIST "+ftpdir, lines.append)
    except:
        print("Error: ftp.retrlines() failed. ftp timeout? Reconnect!")
        return
        
    if len(lines) == 0:
        print("Error: ftp dir is empty")
        return
    
    for line in lines:
#        print(line)
        [ftype, fsize, fname] = [line[0:1], int(line[31:42]), line[56:]]
        
        fext = os.path.splitext(fname)[-1]
        
        if fext == ".zip":
            station_id = int(fname.split("_")[2])
        else:
            station_id = -1 
        
        flist.append([station_id, fname, fext, fsize, ftype])
        
        

    df_ftpdir = pd.DataFrame(flist,columns=["station_id", "name", "ext", "size", "type"])
    return(df_ftpdir)

df_ftpdir = gen_df_from_ftp_dir_listing(ftp, ftp_dir)
df_ftpdir.head(10)

,station_id,name,ext,size,type
0,-1,BESCHREIBUNG_obsgermany_climate_hourly_precipi...,.pdf,166317,-
1,-1,DESCRIPTION_obsgermany_climate_hourly_precipit...,.pdf,161348,-
2,-1,RR_Stundenwerte_Beschreibung_Stationen.txt,.txt,295939,-
3,3,stundenwerte_RR_00003_19950901_20110401_hist.zip,.zip,419296,-
4,20,stundenwerte_RR_00020_20040814_20201231_hist.zip,.zip,432124,-
5,44,stundenwerte_RR_00044_20070401_20201231_hist.zip,.zip,354983,-
6,53,stundenwerte_RR_00053_20051001_20201231_hist.zip,.zip,385830,-
7,71,stundenwerte_RR_00071_20041022_20200101_hist.zip,.zip,402875,-
8,73,stundenwerte_RR_00073_20070401_20201231_hist.zip,.zip,357529,-
9,78,stundenwerte_RR_00078_20041101_20201231_hist.zip,.zip,421522,-


In [10]:
#get the list of station ids and corresponding zip file
df_zips = df_ftpdir[df_ftpdir["ext"]==".zip"]
df_zips.set_index("station_id", inplace = True)
df_zips.head(10)

,name,ext,size,type
station_id,,,,
3,stundenwerte_RR_00003_19950901_20110401_hist.zip,.zip,419296,-
20,stundenwerte_RR_00020_20040814_20201231_hist.zip,.zip,432124,-
44,stundenwerte_RR_00044_20070401_20201231_hist.zip,.zip,354983,-
53,stundenwerte_RR_00053_20051001_20201231_hist.zip,.zip,385830,-
71,stundenwerte_RR_00071_20041022_20200101_hist.zip,.zip,402875,-
73,stundenwerte_RR_00073_20070401_20201231_hist.zip,.zip,357529,-
78,stundenwerte_RR_00078_20041101_20201231_hist.zip,.zip,421522,-
87,stundenwerte_RR_00087_20050201_20201231_hist.zip,.zip,403610,-
91,stundenwerte_RR_00091_20040901_20201231_hist.zip,.zip,419182,-


In [11]:
#read the files in ftp
def grabFile(ftpfullname,localfullname):
    try:
        ret = ftp.cwd(".") # A dummy action to check the connection and to provoke an exception if necessary.
        localfile = open(localfullname, 'wb')
        ftp.retrbinary('RETR ' + ftpfullname, localfile.write, 1024)
        localfile.close()
    
    except ftplib.error_perm:
        print("FTP ERROR. Operation not permitted. File not found?")

    except ftplib.error_temp:
        print("FTP ERROR. Timeout.")

    except ConnectionAbortedError:
        print("FTP ERROR. Connection aborted.")
        
station_fname = df_ftpdir[df_ftpdir['name'].str.contains(station_desc_pattern)]["name"].values[0]
print("grab file: " + station_fname + "\nfrom ftp dir: " + ftp_dir)
grabFile(ftp_dir + station_fname, local_station_dir + station_fname)

grab file: RR_Stundenwerte_Beschreibung_Stationen.txt
from ftp dir: /climate_environment/CDC/observations_germany/climate//hourly/precipitation/historical/


In [12]:
#convert station description text file to csv file and change the column names.
def station_desc_txt_to_csv(txtfile, csvfile):
    file = codecs.open(txtfile,"r","utf-8")
    r = file.readline()
    file.close()
    colnames_de = r.split()
    colnames_de
    
    # German-English dictionary
    translate = \
    {'Stations_id':'station_id',
     'von_datum':'date_from',
     'bis_datum':'date_to',
     'Stationshoehe':'altitude',
     'geoBreite': 'latitude',
     'geoLaenge': 'longitude',
     'Stationsname':'name',
     'Bundesland':'state'}
    
    colnames_en = [translate[h] for h in colnames_de]
    txtfile.encode('unicode_escape').strip()
    # Skip the first two rows and set the column names.
    df = pd.read_fwf(txtfile,skiprows=2,names=colnames_en, parse_dates=["date_from","date_to"],index_col = 0,encoding='unicode_escape')
    
    # write CSV file with field separator semicolon
    df.to_csv(csvfile, sep = ";")
    return(df)

basename = os.path.splitext(station_fname)[0]
df_stations = station_desc_txt_to_csv(local_station_dir + station_fname, local_station_dir + basename + ".csv")
df_stations.head()

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
3,1995-09-01,2011-04-01,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
20,2004-08-14,2022-03-01,432,48.9219,9.9129,Abtsgmünd-Untergröningen,Baden-Württemberg
29,2006-01-10,2022-03-01,260,49.7175,10.9101,Adelsdorf (Kläranlage),Bayern
44,2007-04-01,2022-03-01,44,52.9336,8.2370,Großenkneten,Niedersachsen
46,2006-01-03,2022-03-01,325,48.9450,12.4639,Aholfing,Bayern


In [13]:
# select only counties in North Rhine-Westfalen
isNRW = df_stations['state'].str.contains("Nordrhein")

dfNRW = df_stations[isNRW]

#print("Number of stations in NRW: \n", dfNRW.count())
dfNRW

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
3,1995-09-01,2011-04-01,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
216,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
326,2004-07-01,2013-07-08,120,51.7204,8.0577,Beckum-Unterberg,Nordrhein-Westfalen
389,2009-11-01,2022-03-01,436,51.0148,8.4318,"Berleburg, Bad-Arfeld",Nordrhein-Westfalen
390,2004-07-01,2022-03-01,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
...,...,...,...,...,...,...,...
14184,2016-06-01,2022-03-01,126,51.2242,7.1070,Wuppertal-Buchenhofen/Wupper,Nordrhein-Westfalen
14185,2017-08-01,2022-03-01,304,51.6050,8.8175,Lichtenau-Ebbinghausen (HRB),Nordrhein-Westfalen
14186,2017-08-01,2022-03-01,291,51.5319,8.7289,Gollentaler Grund (HRB),Nordrhein-Westfalen


In [14]:
# Add the names of the zip files only to a list. 
local_zip_list = []

station_ids_selected = list(dfNRW.index)

for station_id in station_ids_selected:
    try:
        fname = df_zips["name"][station_id]
        print(fname)
        grabFile(ftp_dir + fname, local_ftp_ts_dir + fname)
        local_zip_list.append(fname)
    except:
        print("WARNING: TS file for key %d not found in FTP directory." % station_id)

stundenwerte_RR_00003_19950901_20110401_hist.zip
stundenwerte_RR_00216_20041001_20201231_hist.zip
stundenwerte_RR_00326_20040701_20130708_hist.zip
stundenwerte_RR_00389_20091101_20201231_hist.zip
stundenwerte_RR_00390_20040701_20201231_hist.zip
stundenwerte_RR_00554_19950901_20201231_hist.zip
stundenwerte_RR_00555_20080101_20181101_hist.zip
stundenwerte_RR_00599_19950901_19990302_hist.zip
stundenwerte_RR_00603_19990303_20201231_hist.zip
stundenwerte_RR_00613_20041101_20201231_hist.zip
stundenwerte_RR_00617_20040601_20201231_hist.zip
stundenwerte_RR_00644_20050101_20201231_hist.zip
stundenwerte_RR_00796_20041101_20201231_hist.zip
stundenwerte_RR_00871_20050801_20201231_hist.zip
stundenwerte_RR_00902_20061001_20201231_hist.zip
stundenwerte_RR_00934_20041001_20201231_hist.zip
stundenwerte_RR_00989_20050201_20201231_hist.zip
stundenwerte_RR_01024_20060801_20201231_hist.zip
stundenwerte_RR_01046_20041001_20201231_hist.zip
stundenwerte_RR_01078_19950901_20201231_hist.zip
stundenwerte_RR_0124

In [15]:
#generate precipitation time series and store in dataframe
def prec_ts_to_df(fname):
    
    dateparse = lambda dates: [datetime.strptime(str(d), '%Y%m%d%H') for d in dates]

    df = pd.read_csv(fname, delimiter=";", encoding="utf8", index_col="MESS_DATUM", parse_dates = ["MESS_DATUM"], date_parser = dateparse, na_values = [-999.0, -999])

    #df = pd.read_csv(fname, delimiter=";", encoding="iso8859_2",\
    #             index_col="MESS_DATUM", parse_dates = ["MESS_DATUM"], date_parser = dateparse)
    
    # https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd

    # Column headers: remove leading blanks (strip), replace " " with "_", and convert to lower case.
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.index.name = df.index.name.strip().lower().replace(' ', '_').replace('(', '').replace(')', '')
    return(df)

In [16]:
#merge all the precipitation time series generated into singe dataframe
def prec_ts_merge():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dftmp = prec_ts_to_df(myfile)
                s = dftmp["r1"].rename(dftmp["stations_id"][0]).to_frame()
                # outer merge.
                df = pd.merge(df, s, left_index=True, right_index=True, how='outer')

    #df.index.names = ["year"]
    df.index.rename(name = "time", inplace = True)
    return(df)

df_merged_ts = prec_ts_merge()

Zip archive: ../data/original/DWD//hourly/precipitation/historical/stundenwerte_RR_00003_19950901_20110401_hist.zip
Extract product file: produkt_rr_stunde_19950901_20110401_00003.txt



/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


Zip archive: ../data/original/DWD//hourly/precipitation/historical/stundenwerte_RR_00216_20041001_20201231_hist.zip
Extract product file: produkt_rr_stunde_20041001_20201231_00216.txt



/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


Zip archive: ../data/original/DWD//hourly/precipitation/historical/stundenwerte_RR_00326_20040701_20130708_hist.zip
Extract product file: produkt_rr_stunde_20040701_20130708_00326.txt



...

/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


Zip archive: ../data/original/DWD//hourly/precipitation/historical/stundenwerte_RR_13713_20071101_20201231_hist.zip
Extract product file: produkt_rr_stunde_20071101_20201231_13713.txt



/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


Zip archive: ../data/original/DWD//hourly/precipitation/historical/stundenwerte_RR_15000_20110401_20201231_hist.zip
Extract product file: produkt_rr_stunde_20110401_20201231_15000.txt



/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


In [17]:
#glance the contents of the created dataframe
df_merged_ts.head()

,3,216,326,389,390,554,555,599,603,613,...,7344,7374,7378,13669,13670,13671,13696,13700,13713,15000
time,,,,,,,,,,,,,,,,,,,,,
1995-09-01 00:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-09-01 01:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-09-01 02:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-09-01 03:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-09-01 04:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#join the precipitation station description data with the time series data
def ts_append():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
# TEMPERATURE                dftmp = temp_ts_to_df(myfile)
# PRECIPIATION
                dftmp = prec_ts_to_df(myfile)
                dftmp = dftmp.merge(df_stations,how="inner",left_on="stations_id",right_on="station_id",right_index=True)
#                print(dftmp.head(5))
                df = df.append(dftmp)

    #df.index.names = ["year"]
    #df.index.rename(name = "time", inplace = True)
    return(df)

df_appended_ts = ts_append()

Zip archive: ../data/original/DWD//hourly/precipitation/historical/stundenwerte_RR_00003_19950901_20110401_hist.zip
Extract product file: produkt_rr_stunde_19950901_20110401_00003.txt



/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/308058508.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

Zip archive: ../data/original/DWD//hourly/precipitation/historical/stundenwerte_RR_00216_20041001_20201231_hist.zip
Extract product file: produkt_rr_stunde_20041001_20201231_00216.txt



/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/308058508.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

Zip archive: ../data/original/DWD//hourly/precipitation/historical/stundenwerte_RR_00326_20040701_20130708_hist.zip
Extract product file: produkt_rr_stunde_20040701_20130708_00326.txt



...

/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/308058508.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

Zip archive: ../data/original/DWD//hourly/precipitation/historical/stundenwerte_RR_13713_20071101_20201231_hist.zip
Extract product file: produkt_rr_stunde_20071101_20201231_13713.txt



/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/308058508.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

Zip archive: ../data/original/DWD//hourly/precipitation/historical/stundenwerte_RR_15000_20110401_20201231_hist.zip
Extract product file: produkt_rr_stunde_20110401_20201231_15000.txt



/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/2885908043.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/nv/b5gt_2xs6_b2b50_jwyn74wm0000gn/T/ipykernel_2616/308058508.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

In [19]:
df_appended_ts.head()

,stations_id,qn_8,r1,rs_ind,wrtr,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,,,,,
1995-09-01 00:00:00,3,1,0.0,0.0,NaN,eor,1995-09-01,2011-04-01,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
1995-09-01 01:00:00,3,1,0.0,0.0,NaN,eor,1995-09-01,2011-04-01,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
1995-09-01 02:00:00,3,1,0.0,0.0,NaN,eor,1995-09-01,2011-04-01,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
1995-09-01 03:00:00,3,1,0.0,0.0,NaN,eor,1995-09-01,2011-04-01,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
1995-09-01 04:00:00,3,1,0.0,0.0,NaN,eor,1995-09-01,2011-04-01,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen


In [20]:
#only stations in OLPE and Hochsquerlandkrei are selected for the study
df_appended_ts_roi=df_appended_ts[(df_appended_ts['stations_id']==216) |
               (df_appended_ts['stations_id']==1300) |
               (df_appended_ts['stations_id']==2483) |
               (df_appended_ts['stations_id']==2947) |
               (df_appended_ts['stations_id']==3215) |
               (df_appended_ts['stations_id']==4488) |
               (df_appended_ts['stations_id']==5468) |
               (df_appended_ts['stations_id']==6264) |
               (df_appended_ts['stations_id']==7330)]
df_appended_ts_roi

,stations_id,qn_8,r1,rs_ind,wrtr,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,,,,,
2004-10-01 00:00:00,216,1,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2004-10-01 01:00:00,216,1,0.0,0.0,0.0,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2004-10-01 02:00:00,216,1,0.0,0.0,0.0,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2004-10-01 03:00:00,216,1,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2004-10-01 04:00:00,216,1,0.0,0.0,0.0,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.9780,Arnsberg-Neheim,Nordrhein-Westfalen
2020-12-31 20:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.9780,Arnsberg-Neheim,Nordrhein-Westfalen
2020-12-31 21:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.9780,Arnsberg-Neheim,Nordrhein-Westfalen


In [21]:
#the date of interest is selected 2018-04-16 to 2018-08-16
idx=(df_appended_ts_roi.index >= '2018-04-16 00:00:00') & (df_appended_ts_roi.index<'2018-08-16 00:00:00')

In [22]:
df_appended_ts_countiesofinterest=df_appended_ts_roi[idx]
df_appended_ts_countiesofinterest.tail()

,stations_id,qn_8,r1,rs_ind,wrtr,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,,,,,
2018-08-15 19:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.978,Arnsberg-Neheim,Nordrhein-Westfalen
2018-08-15 20:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.978,Arnsberg-Neheim,Nordrhein-Westfalen
2018-08-15 21:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.978,Arnsberg-Neheim,Nordrhein-Westfalen
2018-08-15 22:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.978,Arnsberg-Neheim,Nordrhein-Westfalen
2018-08-15 23:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.978,Arnsberg-Neheim,Nordrhein-Westfalen


In [23]:
df_appended_ts_countiesofinterest

,stations_id,qn_8,r1,rs_ind,wrtr,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,,,,,
2018-04-16 00:00:00,216,3,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2018-04-16 01:00:00,216,3,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2018-04-16 02:00:00,216,3,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2018-04-16 03:00:00,216,3,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2018-04-16 04:00:00,216,3,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-15 19:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.9780,Arnsberg-Neheim,Nordrhein-Westfalen
2018-08-15 20:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.9780,Arnsberg-Neheim,Nordrhein-Westfalen
2018-08-15 21:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.9780,Arnsberg-Neheim,Nordrhein-Westfalen


In [24]:
df_appended_ts_countiesofinterest

,stations_id,qn_8,r1,rs_ind,wrtr,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,,,,,
2018-04-16 00:00:00,216,3,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2018-04-16 01:00:00,216,3,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2018-04-16 02:00:00,216,3,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2018-04-16 03:00:00,216,3,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
2018-04-16 04:00:00,216,3,0.0,0.0,NaN,eor,2004-10-01,2022-02-25,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-15 19:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.9780,Arnsberg-Neheim,Nordrhein-Westfalen
2018-08-15 20:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.9780,Arnsberg-Neheim,Nordrhein-Westfalen
2018-08-15 21:00:00,7330,3,0.0,0.0,NaN,eor,2005-10-01,2022-03-01,159,51.4633,7.9780,Arnsberg-Neheim,Nordrhein-Westfalen


In [25]:
#date vs station id data
df_appended_ts_countiesofinterest=df_appended_ts_countiesofinterest.pivot_table(index="mess_datum",columns='stations_id',   values='r1')
df_appended_ts_countiesofinterest

stations_id,216,1300,2483,2947,3215,4488,5468,6264,7330
mess_datum,,,,,,,,,
2018-04-16 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-04-16 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-04-16 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-04-16 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-04-16 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2018-08-15 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-15 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-15 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:

#df_appended_ts_countiesofinterest.reset_index(level=0, inplace=True)
#df_appended_ts_countiesofinterest.rename(columns={'mess_datum': 'ts'}, inplace=True)
#df_appended_ts_countiesofinterest.to_csv(r'data_file.csv', index = False)
df_appended_ts_countiesofinterest

stations_id,ts,216,1300,2483,2947,3215,4488,5468,6264,7330
0,2018-04-16 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-04-16 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-04-16 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-04-16 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-04-16 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2923,2018-08-15 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2924,2018-08-15 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2925,2018-08-15 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2926,2018-08-15 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
#obtain the daily precipitation sum value from the hourly values.
#df_appended_ts_countiesofinterest_sum=df_appended_ts_countiesofinterest.groupby(df_appended_ts_countiesofinterest.index.strftime("%y/%m/%d")).sum()
#df_appended_ts_countiesofinterest_sum.reset_index(level=0, inplace=True)
#df_appended_ts_countiesofinterest.rename(columns={'mess_datum': 'ts'}, inplace=True)
#df_appended_ts_countiesofinterest.reset_index(level=0, inplace=True)
#df_appended_ts_countiesofinterest_sum.rename(columns={'mess_datum': 'ts'}, inplace=True)
#df_appended_ts_countiesofinterest_sum.to_csv(r'sum_data_file.csv', index = False)
#df_appended_ts_countiesofinterest_sum